<a href="https://colab.research.google.com/github/durlabhjilegend-lgtm/SuitsEncoder/blob/main/JurisAI(final).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import os
os.environ["GEMINI_API_KEY"] = "GOOGLE_API_KEY"
!pip install django djangorestframework google-cloud-documentai google-generativeai PyPDF2 python-decouple django-cors-headers pyngrok streamlit --quiet


In [16]:
!rm -rf jurisai_backend
!django-admin startproject jurisai_backend
%cd jurisai_backend
!python manage.py startapp api
open('api/__init__.py', 'a').close()


/content/jurisai_backend


In [17]:
settings_code = """
import os
from pathlib import Path

BASE_DIR = Path(__file__).resolve().parent.parent

SECRET_KEY = 'colab-jurisai-secret'
DEBUG = True
ALLOWED_HOSTS = ['*']

INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    'rest_framework',
    'corsheaders',
    'api',
]

MIDDLEWARE = [
    'corsheaders.middleware.CorsMiddleware',
    'django.middleware.common.CommonMiddleware',
    'django.middleware.security.SecurityMiddleware',
    'django.contrib.sessions.middleware.SessionMiddleware',
    'django.middleware.csrf.CsrfViewMiddleware',
    'django.contrib.auth.middleware.AuthenticationMiddleware',
    'django.contrib.messages.middleware.MessageMiddleware',
    'django.middleware.clickjacking.XFrameOptionsMiddleware',
]

ROOT_URLCONF = 'jurisai_backend.urls'

TEMPLATES = [
    {
        'BACKEND': 'django.template.backends.django.DjangoTemplates',
        'DIRS': [],
        'APP_DIRS': True,
        'OPTIONS': {
            'context_processors': [
                'django.template.context_processors.debug',
                'django.template.context_processors.request',
                'django.contrib.auth.context_processors.auth',
                'django.contrib.messages.context_processors.messages',
            ],
        },
    },
]

DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.sqlite3',
        'NAME': BASE_DIR / 'db.sqlite3',
    }
}

REST_FRAMEWORK = {
    'DEFAULT_PERMISSION_CLASSES': [
        'rest_framework.permissions.AllowAny',
    ],
    'DEFAULT_RENDERER_CLASSES': [
        'rest_framework.renderers.JSONRenderer',
    ],
}
CORS_ALLOW_ALL_ORIGINS = True
CORS_ALLOW_CREDENTIALS = True
LANGUAGE_CODE = 'en-us'
TIME_ZONE = 'UTC'
USE_I18N = True
USE_TZ = True

STATIC_URL = '/static/'
MEDIA_URL = '/media/'
MEDIA_ROOT = os.path.join(BASE_DIR, 'media')
"""
with open('jurisai_backend/settings.py', 'w') as f:
    f.write(settings_code)


In [18]:
models_code = """
from django.db import models
import uuid

class LegalDocument(models.Model):
    DOCUMENT_TYPES = [
        ('agreement', 'Agreement'),
        ('contract', 'Contract Vetting'),
        ('privacy_policy', 'Privacy Policy'),
        ('terms_of_service', 'Terms of Service'),
    ]
    id = models.UUIDField(primary_key=True, default=uuid.uuid4, editable=False)
    document_type = models.CharField(max_length=32, choices=DOCUMENT_TYPES)
    file_name = models.CharField(max_length=255)
    upload_date = models.DateTimeField(auto_now_add=True)
    extracted_text = models.TextField(blank=True)
    analysis_result = models.JSONField(null=True, blank=True)
    highlighted_sections = models.JSONField(null=True, blank=True)
    def __str__(self):
        return f"{self.file_name} ({self.get_document_type_display()})"

class DocumentAnalysis(models.Model):
    document = models.OneToOneField(LegalDocument, on_delete=models.CASCADE, related_name='detailed_analysis')
    registration_status = models.CharField(max_length=100, blank=True)
    parties_involved = models.TextField(blank=True)
    effective_date = models.CharField(max_length=255, blank=True)
    purpose = models.TextField(blank=True)
    terms_conditions = models.TextField(blank=True)
    default_clause = models.TextField(blank=True)
    termination_clause = models.TextField(blank=True)
    tenure_period = models.CharField(max_length=255, blank=True)
    owner = models.CharField(max_length=255, blank=True)
    data_collected = models.TextField(blank=True)
    user_rights = models.TextField(blank=True)
    data_retention = models.TextField(blank=True)
    service_terms = models.TextField(blank=True)
    created_at = models.DateTimeField(auto_now_add=True)
    updated_at = models.DateTimeField(auto_now=True)
    def __str__(self):
        return f"Analysis for {self.document.file_name}"

class QuestionAnswer(models.Model):
    document = models.ForeignKey(LegalDocument, on_delete=models.CASCADE, related_name='questions')
    question = models.TextField()
    answer = models.TextField()
    timestamp = models.DateTimeField(auto_now_add=True)
    def __str__(self):
        return f"Q&A [{self.document.file_name}] {self.question[:30]}..."
"""
with open('api/models.py', 'w') as f:
    f.write(models_code)


In [19]:
serializer_code = """
from rest_framework import serializers
from .models import LegalDocument, DocumentAnalysis, QuestionAnswer

class LegalDocumentSerializer(serializers.ModelSerializer):
    class Meta:
        model = LegalDocument
        fields = '__all__'

class DocumentAnalysisSerializer(serializers.ModelSerializer):
    class Meta:
        model = DocumentAnalysis
        fields = '__all__'

class QuestionAnswerSerializer(serializers.ModelSerializer):
    class Meta:
        model = QuestionAnswer
        fields = '__all__'
"""
with open('api/serializers.py', 'w') as f:
    f.write(serializer_code)


In [20]:
views_code = """
from rest_framework import viewsets, status
from rest_framework.decorators import action
from rest_framework.response import Response
from .models import LegalDocument, DocumentAnalysis, QuestionAnswer
from .serializers import LegalDocumentSerializer, DocumentAnalysisSerializer, QuestionAnswerSerializer
import PyPDF2

class LegalDocumentViewSet(viewsets.ModelViewSet):
    queryset = LegalDocument.objects.all()
    serializer_class = LegalDocumentSerializer

    def create(self, request):
        file = request.FILES.get('file')
        document_type = request.data.get('document_type')
        if not file or not document_type:
            return Response({'error': 'File and document_type required.'}, status=status.HTTP_400_BAD_REQUEST)
        pdf_reader = PyPDF2.PdfReader(file)
        text = ""
        for page in pdf_reader.pages:
            text += page.extract_text() or ""
        analysis_result = {"summary": text[:500]}
        highlights = [{"line_number": 1, "text": text.split("\\n")[0], "keyword": "sample", "importance": "high"}]
        doc = LegalDocument.objects.create(
            document_type=document_type,
            file_name=file.name,
            extracted_text=text,
            analysis_result=analysis_result,
            highlighted_sections=highlights
        )
        return Response(self.get_serializer(doc).data, status=status.HTTP_201_CREATED)

    @action(detail=True, methods=['post'])
    def ask_question(self, request, pk=None):
        document = self.get_object()
        question = request.data.get('question')
        import google.generativeai as genai
        import os

        api_key = os.environ.get("GEMINI_API_KEY")
        if not api_key:
            return Response({"answer": "No Gemini API key set."})

        try:
            genai.configure(api_key=api_key)
            model = genai.GenerativeModel('gemini-1.5-flash')
            prompt = f"Based on the following legal document, answer this question: {question}\\n\\nDocument:\\n{document.extracted_text}"
            response = model.generate_content(prompt)
            if hasattr(response, "text"):
                answer = response.text
            elif hasattr(response, "candidates") and len(response.candidates) > 0:
                answer = response.candidates[0]["output"]
            else:
                answer = str(response)
        except Exception as e:
            answer = f"Error calling Gemini: {e}"

        qa = QuestionAnswer.objects.create(document=document, question=question, answer=answer)
        return Response({"question": question, "answer": answer})
"""
with open('api/views.py', 'w') as f:
    f.write(views_code)


In [21]:
api_urls_code = """
from django.urls import path, include
from rest_framework.routers import DefaultRouter
from .views import LegalDocumentViewSet

router = DefaultRouter()
router.register(r'documents', LegalDocumentViewSet)

urlpatterns = [
    path('', include(router.urls)),
]
"""
with open('api/urls.py', 'w') as f:
    f.write(api_urls_code)

main_urls_code = """
from django.contrib import admin
from django.urls import path, include

urlpatterns = [
    path('admin/', admin.site.urls),
    path('api/', include('api.urls')),
]
"""
with open('jurisai_backend/urls.py', 'w') as f:
    f.write(main_urls_code)


In [22]:
admin_code = """
from django.contrib import admin
from .models import LegalDocument, DocumentAnalysis, QuestionAnswer
admin.site.register(LegalDocument)
admin.site.register(DocumentAnalysis)
admin.site.register(QuestionAnswer)
"""
with open('api/admin.py', 'w') as f:
    f.write(admin_code)


In [23]:
!python manage.py makemigrations
!python manage.py migrate


Migrations for 'api':
  api/migrations/0001_initial.py
    + Create model LegalDocument
    + Create model DocumentAnalysis
    + Create model QuestionAnswer
Operations to perform:
  Apply all migrations: admin, api, auth, contenttypes, sessions
Running migrations:
  Applying contenttypes.0001_initial... OK
  Applying auth.0001_initial... OK
  Applying admin.0001_initial... OK
  Applying admin.0002_logentry_remove_auto_add... OK
  Applying admin.0003_logentry_add_action_flag_choices... OK
  Applying api.0001_initial... OK
  Applying contenttypes.0002_remove_content_type_name... OK
  Applying auth.0002_alter_permission_name_max_length... OK
  Applying auth.0003_alter_user_email_max_length... OK
  Applying auth.0004_alter_user_username_opts... OK
  Applying auth.0005_alter_user_last_login_null... OK
  Applying auth.0006_require_contenttypes_0002... OK
  Applying auth.0007_alter_validators_add_error_messages... OK
  Applying auth.0008_alter_user_username_max_length... OK
  Applying auth.0

In [24]:
import os
from pyngrok import ngrok, conf
import threading, time
from google.colab import userdata

# Get the authtoken from Colab secrets
NGROK_AUTH_TOKEN = userdata.get('NGROK_AUTH_TOKEN')

# Configure pyngrok with the authtoken
if NGROK_AUTH_TOKEN:
    conf.get_default().auth_token = NGROK_AUTH_TOKEN
    print("ngrok authtoken configured.")
else:
    print("Warning: NGROK_AUTH_TOKEN not found in Colab secrets. ngrok may not work.")


os.environ.setdefault('DJANGO_SETTINGS_MODULE','jurisai_backend.settings')
import django
django.setup()

def run():
    os.system("python manage.py runserver 0.0.0.0:8000")

t = threading.Thread(target=run)
t.start()

time.sleep(4)
try:
  public_url = ngrok.connect(8000)
  print("Django REST API is live:", public_url)
except Exception as e:
  print(f"Failed to connect ngrok: {e}")
  print("Please ensure you have added your NGROK_AUTH_TOKEN to Colab secrets.")

ngrok authtoken configured.
Django REST API is live: NgrokTunnel: "https://1b15ae1e142a.ngrok-free.app" -> "http://localhost:8000"


In [25]:
import os
os.chdir("..")

streamlit_code = '''
import streamlit as st
import requests

API_URL = "http://localhost:8000/api/documents/"  # <-- Replace with your backend ngrok URL

st.title("⚖️ JurisAI Legal Document Analyzer")

dtype = st.selectbox("Select Document Type", [
    ("agreement", "Agreement"),
    ("contract", "Contract Vetting"),
    ("privacy_policy", "Privacy Policy"),
    ("terms_of_service", "Terms of Service")
], format_func=lambda x: x[1])

pdf = st.file_uploader("Upload PDF", type='pdf')

if pdf and st.button("Analyze"):
    files = {'file': (pdf.name, pdf.getvalue(), 'application/pdf')}
    data = {'document_type': dtype[0]}
    with st.spinner("Analyzing..."):
        response = requests.post(API_URL, files=files, data=data)
        if response.status_code == 201:
            result = response.json()
            st.markdown("**Document ID:** `%s`" % result.get("id", "Not available"))
            st.write("**Analysis Result:**", result.get("analysis_result", {}))
            st.write("**Highlights:**", result.get("highlighted_sections", []))
            st.code(result.get("extracted_text", ""))
        else:
            st.warning(f"Failed: {response.text}")

doc_id = st.text_input("Paste Document ID for Q&A:")
question = st.text_input("Your question:")
if doc_id and question and st.button("Ask"):
    r = requests.post(f"{API_URL}{doc_id}/ask_question/", json={"question": question})
    if r.status_code == 200:
        st.write("Answer:", r.json()["answer"])
    else:
        st.warning("Error from backend: " + r.text)
'''

with open("streamlit_jurisai.py", 'w') as f:
    f.write(streamlit_code)


In [26]:
from pyngrok import ngrok
import subprocess
import time

ngrok.kill()  # stop old tunnels if any

proc = subprocess.Popen(["streamlit", "run", "streamlit_jurisai.py", "--server.port=8501"])
time.sleep(7)
stream_url = ngrok.connect(8501)
print("Streamlit app available at:", stream_url)

#After running all the codes, kindly click the ngrok-free.app link, not the local host

Streamlit app available at: NgrokTunnel: "https://62db90d0bf85.ngrok-free.app" -> "http://localhost:8501"
